In [2]:
# 引入模組
import numpy as np
import pandas as pd
import datetime
import pymysql

def dbConnection():
    # 建立MySQL連線
    dbBase = pymysql.connect(host='140.134.51.26', port=3306, user='root', passwd='Gtml@3777', db='ResourceApp', charset='utf8')
    return dbBase

# 框範圍
def dataResource(dbBase, startDate, endDate):
    sql = "SELECT CR.RoomName, CR.BuildName, CR.Unit, CR.RoomName2, HI.ip, HI.ReceiveTime, HI.hrSWInstalledName, HI.hrSWInstalledDate, now() \
    FROM ResourceApp.ConfRoomList AS CR \
    inner join ResourceApp.DeviceConf AS DC \
    ON CR.RoomName = DC.RoomName \
    INNER JOIN ResourceApp.HrSWInstalled AS HI \
    ON DC.IPAddress1 = HI.ip \
    WHERE HI.ReceiveTime >= str_to_date('{0}', '%Y-%m-%d %H:%i:%s') AND HI.ReceiveTime < str_to_date('{1}', '%Y-%m-%d %H:%i:%s')".format(startDate, endDate)
    # print(sql)
    cursor = dbBase.cursor()
    cursor.execute(sql)
    dataResult = cursor.fetchall()
    return dataResult

def ClassRoomSoftInstalledSQL():
    sql = "INSERT INTO ResourceApp.ClassRoomSoftInstalled (RoomName,BuildName,Unit,RoomName2,ip,ReceiveTime,hrSWInstalledName,ClassRoomSoftInstalled,CreateTime) \
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    return sql

if __name__ == "__main__":
    # 建立連線
    dbBase = dbConnection()
    # 撈範圍
    dataResult = dataResource(dbBase, "2020-08-16", "2020-08-17")
    print(type(dataResult))
    cursor = dbBase.cursor()
    try:
        sqlCMD = ClassRoomSoftInstalledSQL()
        cursor.executemany(sqlCMD, dataResult)
        dbBase.commit()
    except pymysql.Error as e:
        print("Err:", e)
    finally:
        #關閉連線
        cursor.close()
        dbBase.close()

    print("Done!!")



KeyboardInterrupt: 

In [ ]:
SELECT CR.RoomName, CR.BuildName, CR.Unit, CR.RoomName2,II.ip, II.ReceiveTime, II.hrSWInstalledName, II.hrSWInstalledDate, now()
FROM(
SELECT distinct ip, STR_TO_DATE(ReceiveTime, '%Y-%m-%d') as ReceiveTime, hrSWInstalledName, hrSWInstalledDate
FROM ResourceApp.HrSWInstalled AS HI
WHERE HI.ReceiveTime >= '2020-08-15' AND HI.ReceiveTime < '2020-08-16'
) AS II
INNER JOIN  ResourceApp.DeviceConf AS DC
ON II.ip = DC.IPAddress1
INNER JOIN ResourceApp.ConfRoomList AS CR
ON CR.RoomName = DC.RoomName